<a href="https://colab.research.google.com/github/kim1987/aiffel/blob/main/aiffel/deeper/dp_four.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torchtext.datasets import AG_NEWS
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.datasets import reuters

In [3]:
import tensorflow as tf

In [4]:
import pandas as pd
import numpy as np

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [7]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [8]:
def make_dtm(reuters,num_words=10000):
  (x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=num_words, test_split=0.2,maxlen=512)
  word_index = reuters.get_word_index(path="reuters_word_index.json")
  index_to_word = { index+3 : word for word, index in word_index.items() }
  for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
    index_to_word[index]=token

  decoded = []
  for i in range(len(x_train)):
      t = ' '.join([index_to_word[index] for index in x_train[i]])
      decoded.append(t)
  x_train_de = decoded

  decoded = []
  for i in range(len(x_test)):
      t = ' '.join([index_to_word[index] for index in x_test[i]])
      decoded.append(t)
  x_test_de = decoded

  dtmvector = CountVectorizer()
  x_train_dtm = dtmvector.fit_transform(x_train_de)
  x_test_dtm = dtmvector.transform(x_test_de)

  tfidf_transformer = TfidfTransformer()
  tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
  tfidfv_test = tfidf_transformer.transform(x_test_dtm)
  
  return (x_train,tfidfv,y_train),(x_test,tfidfv_test,y_test),(word_index,index_to_word)



In [9]:
def make_my_series(num_words):
  (x_train,tfidfv,y_train),(x_test,tfidfv_test,y_test),_ = make_dtm(reuters,num_words)
  row_data =pd.Series(name=str(num_words))

  models=[MultinomialNB(),ComplementNB(),LogisticRegression(C=10000, penalty='l2'),LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False),
          DecisionTreeClassifier(max_depth=10, random_state=0),RandomForestClassifier(n_estimators=5, random_state=0),GradientBoostingClassifier(random_state=0),
          VotingClassifier(estimators=[('lr', LogisticRegression(C=10000, penalty='l2')),
                                       ('cb', ComplementNB()),
                                       ('grbt', GradientBoostingClassifier(random_state=0))
                                      ], voting='soft', n_jobs=-1)
          ]
  names =['naive','Complement Naive','logistic','Linear SVC','DecisionTree','RandomForest','GradientBoosting','Voting']


  for name,model in zip(names,models):
    m_model= model
    m_model.fit(tfidfv, y_train)
    predicted = m_model.predict(tfidfv_test)
    row_data[name]=accuracy_score(y_test, predicted)
    
  #tensorflow model
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=num_words,
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(46)
    ])
  
  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
  x_train=tf.keras.preprocessing.sequence.pad_sequences(x_train,maxlen=512,padding='post')
  x_test=tf.keras.preprocessing.sequence.pad_sequences(x_test,maxlen=512,padding='post')

  model.fit(x_train,y_train)#,epochs=10,batch_size=64
  loss,accuracy = model.evaluate(x_test,y_test,batch_size=64)
  row_data['lstm'] = accuracy

  model.fit(x_train,y_train,epochs=9,batch_size=64)
  loss,accuracy = model.evaluate(x_test,y_test,batch_size=64)
  row_data['lstm_epoch'] = accuracy

  return row_data

In [10]:
my_chart = pd.DataFrame()

In [11]:
for num_words in [5000,10000,15000,20000]:
  my_chart=my_chart.append(make_my_series(num_words))

565248/550378 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  Converge

34/34 [==============================] - 11s 228ms/step - loss: 2.0213 - accuracy: 0.5088
Epoch 1/9
136/136 [==============================] - 134s 985ms/step - loss: 1.9549 - accuracy: 0.5067
Epoch 2/9
136/136 [==============================] - 134s 983ms/step - loss: 1.8065 - accuracy: 0.5225
Epoch 3/9
136/136 [==============================] - 134s 984ms/step - loss: 1.7021 - accuracy: 0.5615
Epoch 4/9
136/136 [==============================] - 133s 981ms/step - loss: 1.8051 - accuracy: 0.5069
Epoch 5/9
136/136 [==============================] - 134s 983ms/step - loss: 1.7135 - accuracy: 0.5479
Epoch 6/9
136/136 [==============================] - 134s 982ms/step - loss: 1.6687 - accuracy: 0.5627
Epoch 7/9
136/136 [==============================] - 134s 983ms/step - loss: 1.6260 - accuracy: 0.5896
Epoch 8/9
136/136 [==============================] - 133s 981ms/step - loss: 1.6023 - accuracy: 0.5971
Epoch 9/9
34/34 [==============================] - 8s 227ms/step - loss: 1.6745 - accu

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  Converge

34/34 [==============================] - 11s 231ms/step - loss: 2.0406 - accuracy: 0.4445
Epoch 1/9
136/136 [==============================] - 137s 1s/step - loss: 1.8827 - accuracy: 0.4877
Epoch 2/9
136/136 [==============================] - 137s 1s/step - loss: 1.7497 - accuracy: 0.5294
Epoch 3/9
136/136 [==============================] - 137s 1s/step - loss: 1.6687 - accuracy: 0.5707
Epoch 4/9
136/136 [==============================] - 136s 1s/step - loss: 1.6134 - accuracy: 0.5895
Epoch 5/9
136/136 [==============================] - 137s 1s/step - loss: 1.5470 - accuracy: 0.6006
Epoch 6/9
136/136 [==============================] - 136s 1s/step - loss: 1.4945 - accuracy: 0.6155
Epoch 7/9
136/136 [==============================] - 136s 1s/step - loss: 1.4303 - accuracy: 0.6407
Epoch 8/9
136/136 [==============================] - 136s 1s/step - loss: 1.3753 - accuracy: 0.6563
Epoch 9/9
34/34 [==============================] - 8s 236ms/step - loss: 1.4814 - accuracy: 0.6327


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  Converge

34/34 [==============================] - 11s 229ms/step - loss: 2.0077 - accuracy: 0.5190
Epoch 1/9
136/136 [==============================] - 139s 1s/step - loss: 1.9626 - accuracy: 0.5108
Epoch 2/9
136/136 [==============================] - 139s 1s/step - loss: 1.8046 - accuracy: 0.5188
Epoch 3/9
136/136 [==============================] - 139s 1s/step - loss: 1.6761 - accuracy: 0.5687
Epoch 4/9
136/136 [==============================] - 140s 1s/step - loss: 1.6182 - accuracy: 0.5858
Epoch 5/9
136/136 [==============================] - 140s 1s/step - loss: 1.6675 - accuracy: 0.5553
Epoch 6/9
136/136 [==============================] - 139s 1s/step - loss: 1.5761 - accuracy: 0.5923
Epoch 7/9
136/136 [==============================] - 139s 1s/step - loss: 1.5522 - accuracy: 0.6001
Epoch 8/9
136/136 [==============================] - 139s 1s/step - loss: 1.5343 - accuracy: 0.6014
Epoch 9/9
34/34 [==============================] - 8s 243ms/step - loss: 1.6659 - accuracy: 0.5772


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  Converge

34/34 [==============================] - 10s 224ms/step - loss: 2.0840 - accuracy: 0.4570
Epoch 1/9
136/136 [==============================] - 140s 1s/step - loss: 2.0062 - accuracy: 0.4820
Epoch 2/9
136/136 [==============================] - 140s 1s/step - loss: 1.8905 - accuracy: 0.5030
Epoch 3/9
136/136 [==============================] - 140s 1s/step - loss: 1.7552 - accuracy: 0.5412
Epoch 4/9
136/136 [==============================] - 140s 1s/step - loss: 1.6734 - accuracy: 0.5769
Epoch 5/9
136/136 [==============================] - 140s 1s/step - loss: 1.6341 - accuracy: 0.5845
Epoch 6/9
136/136 [==============================] - 140s 1s/step - loss: 1.5808 - accuracy: 0.6013
Epoch 7/9
136/136 [==============================] - 140s 1s/step - loss: 1.5452 - accuracy: 0.6126
Epoch 8/9
136/136 [==============================] - 140s 1s/step - loss: 1.6362 - accuracy: 0.5764
Epoch 9/9
34/34 [==============================] - 8s 236ms/step - loss: 1.7164 - accuracy: 0.5786


In [12]:
print(my_chart)

       Complement Naive  DecisionTree  ...  lstm_epoch     naive
5000           0.777521      0.628122  ...    0.586957  0.679463
10000          0.778908      0.618409  ...    0.632747  0.658649
15000          0.777983      0.620722  ...    0.577243  0.634135
20000          0.776596      0.619796  ...    0.578631  0.623959

[4 rows x 10 columns]


In [13]:
my_chart

,Complement Naive,DecisionTree,GradientBoosting,Linear SVC,RandomForest,Voting,logistic,lstm,lstm_epoch,naive
5000,0.777521,0.628122,0.770120,0.783071,0.683164,0.824699,0.814986,0.508788,0.586957,0.679463
10000,0.778908,0.618409,0.770120,0.781684,0.690102,0.820999,0.820999,0.444496,0.632747,0.658649
15000,0.777983,0.620722,0.770583,0.785384,0.671138,0.825624,0.824699,0.518964,0.577243,0.634135
20000,0.776596,0.619796,0.773358,0.776596,0.681776,0.825162,0.823312,0.456984,0.578631,0.623959


lstm 을 이용한 모델은 vocab 사이즈가 커질수록 학습에 어려움을 느낌.  
lstm 은 다른 전처리 과정을 거침. TF-IDF Matrix을 사용하지 않음.  
에폭이 상승할 수록 학습이 되는 것을 보였으나 에폭을 10회정도만 사용.   
및 하이퍼파라미터의 변화를 주지 않음